In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold, cross_validate, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay


In [2]:
def training(X, y, model, cv=5):
    '''
    Train a model using cross-validation and display the confusion matrix.
    '''
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    scores = cross_validate(model, X, y, cv=kf, scoring=['accuracy'])
    y_pred = cross_val_predict(model, X, y, cv=kf)
    cm = confusion_matrix(y, y_pred)
    # disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    # disp.plot()
    # plt.show()
    return scores

In [3]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test-full.csv')
X = train.drop(columns=['Id', 'Cover_Type'])

X_test = test.drop(columns=['Id'])

original_features = X.columns.tolist()
y = train['Cover_Type']
id = train['Id']
id_test = test['Id']

model = model = LGBMClassifier(
    random_state=42,
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1
)

training(X, y, model)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2161
[LightGBM] [Info] Number of data points in the train set: 12096, number of used features: 44
[LightGBM] [Info] Start training from score -1.943021
[LightGBM] [Info] Start training from score -1.964602
[LightGBM] [Info] Start training from score -1.927561
[LightGBM] [Info] Start training from score -1.948228
[LightGBM] [Info] Start training from score -1.957552
[LightGBM] [Info] Start training from score -1.936120
[LightGBM] [Info] Start training from score -1.944753
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2156
[LightGBM

{'fit_time': array([2.766155  , 2.7596972 , 2.05447006, 2.13185501, 2.13816786]),
 'score_time': array([0.09045601, 0.09301877, 0.09667706, 0.08787107, 0.09159398]),
 'test_accuracy': array([0.87136243, 0.86640212, 0.8640873 , 0.87202381, 0.86706349])}

In [4]:
X['Elevation_x_Rawah'] = X['Elevation'] * X['Wilderness_Area1']
X['Elevation_x_Neota'] = X['Elevation'] * X['Wilderness_Area2'] 
X['Elevation_x_Comanche'] = X['Elevation'] * X['Wilderness_Area3']
X['Elevation_x_Cache'] = X['Elevation'] * X['Wilderness_Area4']

X_test['Elevation_x_Rawah'] = X_test['Elevation'] * X_test['Wilderness_Area1']
X_test['Elevation_x_Neota'] = X_test['Elevation'] * X_test['Wilderness_Area2'] 
X_test['Elevation_x_Comanche'] = X_test['Elevation'] * X_test['Wilderness_Area3']
X_test['Elevation_x_Cache'] = X_test['Elevation'] * X_test['Wilderness_Area4']

ecologically_inspired_features = ['Elevation_x_Rawah', 'Elevation_x_Neota', 'Elevation_x_Comanche', 'Elevation_x_Cache']
# training(X[original_features + ecologically_inspired_features], y, model)

In [5]:
# Euclidean distance to water (hypotenuse of horizontal and vertical)
X['Euclidean_Distance_To_Hydrology'] = np.sqrt(
    X['Horizontal_Distance_To_Hydrology']**2 + 
    X['Vertical_Distance_To_Hydrology']**2
)

# Water presence flag (areas near water)
X['Near_Water'] = (X['Horizontal_Distance_To_Hydrology'] < 100) & \
                   (abs(X['Vertical_Distance_To_Hydrology']) < 20)
            
            
X_test['Euclidean_Distance_To_Hydrology'] = np.sqrt(
    X_test['Horizontal_Distance_To_Hydrology']**2 + 
    X_test['Vertical_Distance_To_Hydrology']**2
)

# Water presence flag (areas near water)
X_test['Near_Water'] = (X_test['Horizontal_Distance_To_Hydrology'] < 100) & \
                   (abs(X_test['Vertical_Distance_To_Hydrology']) < 20)
                   
                                      
distance_to_water_features = ['Euclidean_Distance_To_Hydrology', 'Near_Water']
# training(X[original_features + distance_to_water_features], y, model)

In [6]:
# Roughness - difference between hillshade times
X['Hillshade_Range'] = X['Hillshade_3pm'] - X['Hillshade_9am']
X['Hillshade_Variance'] = X[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].var(axis=1)

# Slope-aspect interaction (northness/eastness)
X['Northness'] = np.cos(np.radians(X['Aspect']))
X['Eastness'] = np.sin(np.radians(X['Aspect']))
X['Slope_x_Northness'] = X['Slope'] * X['Northness']

# Roughness - difference between hillshade times
X_test['Hillshade_Range'] = X_test['Hillshade_3pm'] - X_test['Hillshade_9am']
X_test['Hillshade_Variance'] = X_test[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].var(axis=1)

# Slope-aspect interaction (northness/eastness)
X_test['Northness'] = np.cos(np.radians(X['Aspect']))
X_test['Eastness'] = np.sin(np.radians(X['Aspect']))
X_test['Slope_x_Northness'] = X_test['Slope'] * X_test['Northness']

terrain_features = ['Hillshade_Range', 'Hillshade_Variance', 'Slope_x_Northness']
# training(X[original_features + terrain_features], y, model)

In [7]:
# Roads vs Fire points ratio (different access patterns)
X['Road_to_Fire_Ratio'] = X['Horizontal_Distance_To_Roadways'] / \
                           (X['Horizontal_Distance_To_Fire_Points'] + 1)

# Hydrology to Roads ratio
X['Hydro_to_Roads_Ratio'] = X['Horizontal_Distance_To_Hydrology'] / \
                             (X['Horizontal_Distance_To_Roadways'] + 1)

# Roads vs Fire points ratio (different access patterns)
X_test['Road_to_Fire_Ratio'] = X_test['Horizontal_Distance_To_Roadways'] / \
                           (X_test['Horizontal_Distance_To_Fire_Points'] + 1)

# Hydrology to Roads ratio
X_test['Hydro_to_Roads_Ratio'] = X_test['Horizontal_Distance_To_Hydrology'] / \
                             (X_test['Horizontal_Distance_To_Roadways'] + 1)
                             
distance_ratio_features = ['Road_to_Fire_Ratio', 'Hydro_to_Roads_Ratio']
# # training(X[original_features + distance_ratio_features], y, model)

In [8]:
# Create elevation bands based on ecological knowledge
def elevation_zone(elev):
    if elev < 2750:
        return 'low'  # Ponderosa, Douglas-fir
    elif elev < 3000:
        return 'mid_low'  # Lodgepole
    elif elev < 3250:
        return 'mid_high'  # Spruce/Fir transition
    else:
        return 'high'  # Spruce/Fir, Krummholz

X['Elevation_Zone'] = X['Elevation'].apply(elevation_zone)
X = pd.get_dummies(X, columns=['Elevation_Zone'])

X_test['Elevation_Zone'] = X_test['Elevation'].apply(elevation_zone)
X_test = pd.get_dummies(X_test, columns=['Elevation_Zone'])

eco_zones = [col for col in X.columns if col.startswith('Elevation_Zone_')]
# training(X[original_features + eco_zones], y, model)

In [9]:
import re

soil_climatic_zones = {
    1: 'lower_montane_dry', 2: 'lower_montane', 3: 'montane_dry',
    4: 'montane', 5: 'montane_mixed', 6: 'montane_subalpine',
    7: 'subalpine', 8: 'alpine'
}

def soil_idx(col):
    m = re.search(r'(\d+)$', col)   # last digits
    return int(m.group(1)) if m else None

soil_cols_all = [c for c in X.columns if c.startswith('Soil_Type')]

for zone_num, zone_name in soil_climatic_zones.items():
    zone_cols = [
        c for c in soil_cols_all
        if (soil_idx(c) is not None) and (soil_idx(c) // 1000 == zone_num)
    ]
    if zone_cols:
        X[f'Climatic_Zone_{zone_name}'] = X[zone_cols].any(axis=1).astype(int)

soil_type_features = [c for c in X.columns if c.startswith('Climatic_Zone_')]

# training(X[original_features + soil_type_features], y, model)

In [10]:
# High elevation + specific aspects (Spruce/Fir prefer north aspects)
X['High_North'] = (X['Elevation'] > 3000) & (X['Northness'] > 0.5)

# Distance to roads + fire points (managed vs natural areas)
X['Remote_Index'] = (X['Horizontal_Distance_To_Roadways'] + 
                      X['Horizontal_Distance_To_Fire_Points']) / 2

# Slope position index
X['Slope_Position'] = X['Elevation'] * X['Slope'] / 1000

# High elevation + specific aspects (Spruce/Fir prefer north aspects)
X_test['High_North'] = (X_test['Elevation'] > 3000) & (X_test['Northness'] > 0.5)

# Distance to roads + fire points (managed vs natural areas)
X_test['Remote_Index'] = (X_test['Horizontal_Distance_To_Roadways'] + 
                      X_test['Horizontal_Distance_To_Fire_Points']) / 2

# Slope position index
X_test['Slope_Position'] = X_test['Elevation'] * X_test['Slope'] / 1000

complex_features = ['High_North', 'Remote_Index', 'Slope_Position']
# training(X[original_features + complex_features], y, model)

In [11]:
# from sklearn.preprocessing import PolynomialFeatures

# # Select features that show some correlation
# key_features = ['Elevation', 'Aspect', 'Slope', 
#                 'Horizontal_Distance_To_Hydrology',
#                 'Horizontal_Distance_To_Roadways']

# # Create interaction terms up to degree 2
# poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
# poly_features = poly.fit_transform(X[key_features])

# poly_feature_names = poly.get_feature_names_out(key_features)
# poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)
# X_poly = pd.concat([X.reset_index(drop=True), poly_df], axis=1)
# training(X_poly[original_features + list(poly_feature_names)], y, model)

In [12]:
# testing the iteraction of the different groups and getting the best combination of features

all_features = original_features + ecologically_inspired_features + distance_to_water_features + terrain_features + distance_ratio_features + eco_zones + soil_type_features + complex_features #+ list(poly_feature_names)
# training(X_poly[all_features], y, model)

In [13]:
def prune_features(X, y, features, train_func, model, tol=0.0005):
    '''
    Function to iteratively prune features based on their importance and the impact on model performance.
     - X: feature dataframe
     - y: target variable
     - features: list of features to consider
     - train_func: function to train the model and return performance metric
     - model: the model to train
     - tol: tolerance for performance drop to consider a feature as non-essential
     Returns a pruned list of features that contribute most to the model's performance.
    '''
    baseline_scores = train_func(X[features], y, model)
    baseline_score = np.mean(baseline_scores['test_accuracy'])
    pruned_features = features.copy()
    
    for feature in features:
        temp_features = [f for f in pruned_features if f != feature]
        temp_scores = train_func(X[temp_features], y, model)
        temp_score = np.mean(temp_scores['test_accuracy'])
        
        # If removing the feature does not significantly drop performance, prune it
        if baseline_score - temp_score < tol:
            pruned_features.remove(feature)
            print(f"Pruned {feature}: baseline {baseline_score:.4f} -> {temp_score:.4f}")
            baseline_score = temp_score  # Update baseline score for next iteration
        else:
            print(f"Kept {feature}: baseline {baseline_score:.4f} -> {temp_score:.4f}")
    
    return pruned_features
    
pruned_features = prune_features(X, y, all_features, training, model, tol=0.0005)

# training(X[pruned_features], y, model)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5132
[LightGBM] [Info] Number of data points in the train set: 12096, number of used features: 62
[LightGBM] [Info] Start training from score -1.943021
[LightGBM] [Info] Start training from score -1.964602
[LightGBM] [Info] Start training from score -1.927561
[LightGBM] [Info] Start training from score -1.948228
[LightGBM] [Info] Start training from score -1.957552
[LightGBM] [Info] Start training from score -1.936120
[LightGBM] [Info] Start training from score -1.944753
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000447 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5124
[LightGBM] [Info] Number of data points in the train set: 12096, number of used features: 62
[LightGBM] [Info] Start training from score 

In [14]:
pruned_features

['Slope',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type10',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type26',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type35',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'Elevation_x_Rawah',
 'Elevation_x_Neota',
 'Elevation_x_Comanche',
 'Elevation_x_Cache',
 'Euclidean_Distance_To_Hydrology',
 'Near_Water',
 'Hillshade_Range',
 'Slope_x_Northness',
 'Road_to_Fire_Ratio',
 'Hydro_to_Roads_Ratio',
 'Elevation_Zone_high',
 'Elevation_Zone_mid_high',
 'High_North',
 'Remote_Index',
 '

In [ ]:
df = pd.DataFrame(pruned_features, columns=["features"])
df.to_csv("engineered_features.csv", index=False)

In [17]:
X['Cover_Type'] = y
X['Id'] = id
X[['Id'] + pruned_features + ['Cover_Type']].to_csv('new_training', index=False)

X_test['Id'] = id_test
X_test[['Id'] + pruned_features].to_csv('new_test', index=False)